# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [70]:
import requests
import re
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

### 先搜尋全部的電影代號(ID)資訊

In [71]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'
doc = pq(resp.text)
# soup = BeautifulSoup(resp.text, 'lxml')
# html = soup.find("select", attrs={'name':'movie_id'})
# movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})
# for p in movie_item:
#     print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

In [72]:
# 查看目前上映那些電影，並擷取出其ID資訊
doc1 = doc('select'  '[name="movie_id"]')
movie_item = []
values = []
for i in doc1("option[data-name]"):
    movie_item.append(i.attrib['data-name'])
    movie_item.append(i.attrib['value'])
    print("Movie: %s, ID: %s" % (i.attrib['data-name'], i.attrib['value']))

len(movie_item)

Movie: 大監獄行動, ID: 10450
Movie: 惡靈20, ID: 10444
Movie: 星星知我心, ID: 10409
Movie: Cunningham 機遇之舞, ID: 10385
Movie: 逃出天堂島, ID: 10384
Movie: 鬼眼逼人, ID: 10379
Movie: 動畫電影版 我想吃掉你的胰臟, ID: 10377
Movie: 魯邦三世 The First, ID: 10355
Movie: 重磅腥聞, ID: 10320
Movie: 菠蘿蜜, ID: 10412
Movie: 陰兒, ID: 10393
Movie: 這一次不再錯過你, ID: 10389
Movie: 別告訴她, ID: 10382
Movie: 鬼屋實錄：惡魔之家, ID: 10378
Movie: 驅魔直播, ID: 10375
Movie: 真愛裁會贏, ID: 10362
Movie: 那一夜, ID: 10345
Movie: 兔嘲男孩, ID: 10283
Movie: 海上鋼琴師4K數位修復, ID: 10425
Movie: 鹿皮奇談, ID: 10420
Movie: 大約在冬季, ID: 10408
Movie: 倒數計死, ID: 10390
Movie: 竊竊屍語, ID: 10365
Movie: 舞力決戰, ID: 10344
Movie: 潛艦追緝, ID: 10333
Movie: 霹靂嬌娃, ID: 10124
Movie: 白頭山：半島浩劫, ID: 10354
Movie: CATS貓, ID: 10166
Movie: 海上花, ID: 10376
Movie: 茱蒂, ID: 10112
Movie: 燃燒女子的畫像, ID: 10069
Movie: 葉問4：完結篇, ID: 9890
Movie: STAR WARS : 天行者的崛起, ID: 10225
Movie: 祭旗, ID: 10373
Movie: 黑熊來了, ID: 10357
Movie: 假面騎士劇場版ZI-O Over Quartzer, ID: 10331
Movie: 8級警戒, ID: 10323
Movie: 大說謊家, ID: 10122
Movie: 呆萌特務, ID: 10264
Movie: 去年聖誕節, I

188

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [73]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10225

In [74]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 彰化, 代號(area_id): 22
放映地區: 南投, 代號(area_id): 13
放映地區: 雲林, 代號(area_id): 19
放映地區: 嘉義, 代號(area_id): 21
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17
放映地區: 基隆, 代號(area_id): 18
放映地區: 宜蘭, 代號(area_id): 11
放映地區: 花蓮, 代號(area_id): 12
放映地區: 台東, 代號(area_id): 9
放映地區: 金門, 代號(area_id): 24
放映地區: 澎湖, 代號(area_id): 23


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [75]:
# 指定放映地區
area_id = 28

In [76]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

# soup = BeautifulSoup(resp.text, 'lxml')
# movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})
doc = pq(resp.text)
items = doc('label[for^="date_"]')
for item in items:
    print(item.find('p').text,sep=' ',end='')
    print(item.find('h3').text)
# 列印播放日期
# for date in movie_date:
#     print("%s %s" % (date.p.string, date.h3.string))

一月11
一月12
一月13
一月14
一月15


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [77]:
# 選定要觀看的日期
date = "2020-01-12"

In [78]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [ ]:
'''

  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
  
  Your code here.

'''


In [89]:
doc1 = pq(str(html[0]))

In [104]:
# doc1('ul#theater_id_32.area_time._c.jq_area_time').text()
for inp in doc1('input'):
    print(pq(inp).attr('id'),pq(inp).attr('value'))

72735117 2020-01-12 09:00:00
72735120 2020-01-12 14:30:00
72735123 2020-01-12 20:10:00
72735125 2020-01-12 23:00:00
72735118 2020-01-12 11:45:00
72735119 2020-01-12 12:30:00
72735121 2020-01-12 15:20:00
72735122 2020-01-12 18:10:00
72735124 2020-01-12 21:00:00
72735126 2020-01-12 23:50:00
